In [32]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"     # to keep all output without use of print()

In [33]:
#Read in Auction table
import pandas as pd

Auction = pd.read_csv('C:/Users/AZhen/Downloads/MGT 4058 Project Data - Auction.csv')

In [34]:
#Cleaning the excess columns that I used for notes in the original spreadsheet
Auction.head()

del Auction["RNG"]
del Auction["Unnamed: 10"]
del Auction["Unnamed: 11"]
del Auction["Unnamed: 12"]

,SP Combo,Buyer ID,Seller ID,Product ID,Bid Date,Bid Time,Bid Price,RNG,Ask Price,BSP Combo,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,S-3319-002,B-9220,S-3319,2,11/12/2022,22:03:49,$145,$238,$238,9220/S-3319-002,NaN,Unique Pairs,NaN
1,S-8697-018,B-4413,S-8697,18,11/25/2022,09:29:18,$121,$151,$178,4413/S-8697-018,NaN,48,NaN
2,S-3319-009,B-0150,S-3319,9,11/5/2022,08:58:45,$322,$322,$322,150/S-3319-009,NaN,AKA,Max number of Items being bid on
3,S-1944-044,B-8027,S-1944,44,11/11/2022,22:34:39,$83,$104,$132,8027/S-1944-044,NaN,NaN,NaN
4,S-2488-037,B-8322,S-2488,37,11/25/2022,06:57:27,$145,$175,$179,8322/S-2488-037,NaN,avg 3 bids per item,NaN


In [35]:
#Ensuring that all 150 rows have been correctly loaded
print(Auction.shape)

(150, 9)


### Notes:
* SP Combo is a TEMPORARY column used as an identifier to match and link tables and therefore link winning bids.
* If someone is bidding on the same PRODUCT from the same SELLER, it will have the same "SP Combo" identifier.
* The maximum bid price for a specific combo is the winning bid. 
* From there, the Final Sell Price will go to the Items table, and the 
Buyer ID will go to the Orders table.

In [36]:
#Read in Items table and drop excess columns that were used for notes

Items = pd.read_csv('C:/Users/AZhen/Downloads/MGT 4058 Project Data - Item Data.csv')
Items = Items.drop(['Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'], axis=1)
Items.head()

,Seller ID,Product ID,SP Combo,Item ID,Size,Order ID (O),Final Sell Price (O)
0,S-1442,36.0,S-1442-036,54.0,7,NaN,NaN
1,S-5393,33.0,S-5393-033,49.0,4,NaN,NaN
2,S-9229,35.0,S-9229-035,73.0,7,NaN,NaN
3,S-4532,19.0,S-4532-019,7.0,4,NaN,NaN
4,S-6313,8.0,S-6313-008,94.0,12,NaN,NaN


In [37]:
#When I generated rows with RNG in Google Sheets, it is an uncertain number of items.
#Therefore, rows containing more than 2 NA's (other than Order ID and Sell Price) will be dropped.

narows = Items[Items.isna().any(axis = 1)]
narows
len(narows)

for i in narows.index:
    if sum(narows.loc[i,:].isna()) > 2:
        Items = Items.drop(axis = 0, index = i)


#Check final table size after dropping NA rows
print(Items.shape)

,Seller ID,Product ID,SP Combo,Item ID,Size,Order ID (O),Final Sell Price (O)
0,S-1442,36.0,S-1442-036,54.0,7,NaN,NaN
1,S-5393,33.0,S-5393-033,49.0,4,NaN,NaN
2,S-9229,35.0,S-9229-035,73.0,7,NaN,NaN
3,S-4532,19.0,S-4532-019,7.0,4,NaN,NaN
4,S-6313,8.0,S-6313-008,94.0,12,NaN,NaN
...,...,...,...,...,...,...,...
87,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,NaN,NaN,NaN,NaN,NaN,NaN,NaN


92

(75, 7)


In [44]:
#Extracting SP (Seller-Product) Combo columns to compare
ItemSP = Items["SP Combo"].values.tolist()
AuctionSP = Auction["SP Combo"].values.tolist()

# Number of items that exist but did not get bids
print(len(set(ItemSP)-set(AuctionSP)))

#Create clone to preserve original data
Items2 = Items.copy()

26


In [83]:
#Iterate through each SP Combo in Auction table
#If a specific SP Combo occurs more than once, find maximum

for i in range(0,len(AuctionSP)):
    if AuctionSP.count(AuctionSP[i]) > 1:
        print("Multiple Bids", i)
        Items2[Items["SP Combo"] == AuctionSP[i]] = max(Auction[Auction["SP Combo"] == AuctionSP[i]]["Bid Price"])
    else:
        print("Max", i)

#printing to check that all 150 bids have been accounted for

Multiple Bids 0
Multiple Bids 1
Multiple Bids 2
Multiple Bids 3
Multiple Bids 4
Multiple Bids 5
Multiple Bids 6
Multiple Bids 7
Multiple Bids 8
Multiple Bids 9
Multiple Bids 10
Multiple Bids 11
Multiple Bids 12
Multiple Bids 13
Multiple Bids 14
Multiple Bids 15
Multiple Bids 16
Multiple Bids 17
Multiple Bids 18
Multiple Bids 19
Multiple Bids 20
Multiple Bids 21
Multiple Bids 22
Multiple Bids 23
Multiple Bids 24
Multiple Bids 25
Multiple Bids 26
Multiple Bids 27
Multiple Bids 28
Multiple Bids 29
Multiple Bids 30
Multiple Bids 31
Multiple Bids 32
Multiple Bids 33
Multiple Bids 34
Multiple Bids 35
Multiple Bids 36
Multiple Bids 37
Multiple Bids 38
Multiple Bids 39
Multiple Bids 40
Multiple Bids 41
Multiple Bids 42
Max 43
Multiple Bids 44
Multiple Bids 45
Multiple Bids 46
Multiple Bids 47
Multiple Bids 48
Multiple Bids 49
Multiple Bids 50
Multiple Bids 51
Multiple Bids 52
Multiple Bids 53
Multiple Bids 54
Multiple Bids 55
Multiple Bids 56
Multiple Bids 57
Multiple Bids 58
Multiple Bids 59


In [82]:
#Iterate through bid prices and remove $ signs (generated bids were strings)

bidlist = Auction["Bid Price"].values.tolist()
for i in range (0,len(bidlist)):
    print(i)
    bidlist[i] = bidlist[i][1:len(bidlist[i])]
    bidlist[i] = int(bidlist[i])
bidlist

#Replace "Bid Price" column with cleaned integer bids
Auction["Bid Price"] = bidlist
Auction

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149


[145,
 121,
 322,
 83,
 145,
 174,
 117,
 123,
 247,
 322,
 125,
 172,
 128,
 160,
 258,
 249,
 114,
 113,
 119,
 162,
 109,
 343,
 157,
 146,
 119,
 212,
 300,
 131,
 98,
 98,
 348,
 301,
 100,
 292,
 216,
 195,
 153,
 158,
 115,
 85,
 76,
 192,
 142,
 173,
 145,
 128,
 261,
 123,
 75,
 136,
 338,
 111,
 143,
 122,
 127,
 353,
 122,
 375,
 191,
 241,
 136,
 168,
 92,
 231,
 119,
 150,
 164,
 122,
 182,
 136,
 146,
 317,
 359,
 106,
 160,
 298,
 201,
 218,
 265,
 250,
 259,
 148,
 82,
 133,
 154,
 102,
 315,
 161,
 136,
 68,
 120,
 124,
 143,
 238,
 121,
 308,
 182,
 117,
 203,
 93,
 228,
 120,
 91,
 143,
 147,
 146,
 319,
 109,
 118,
 200,
 243,
 135,
 328,
 125,
 93,
 249,
 107,
 71,
 231,
 221,
 220,
 149,
 275,
 233,
 250,
 224,
 118,
 168,
 139,
 222,
 150,
 94,
 165,
 388,
 363,
 148,
 207,
 127,
 111,
 124,
 297,
 131,
 95,
 306,
 84,
 199,
 153,
 252,
 105,
 121]

,SP Combo,Buyer ID,Seller ID,Product ID,Bid Date,Bid Time,Bid Price,Ask Price,BSP Combo
0,S-3319-002,B-9220,S-3319,2,11/12/2022,22:03:49,145,$238,9220/S-3319-002
1,S-8697-018,B-4413,S-8697,18,11/25/2022,09:29:18,121,$178,4413/S-8697-018
2,S-3319-009,B-0150,S-3319,9,11/5/2022,08:58:45,322,$322,150/S-3319-009
3,S-1944-044,B-8027,S-1944,44,11/11/2022,22:34:39,83,$132,8027/S-1944-044
4,S-2488-037,B-8322,S-2488,37,11/25/2022,06:57:27,145,$179,8322/S-2488-037
...,...,...,...,...,...,...,...,...,...
145,S-7929-010,B-3990,S-7929,10,11/15/2022,09:57:29,199,$245,3990/S-7929-010
146,S-8697-018,B-3943,S-8697,18,11/20/2022,17:52:24,153,$178,3943/S-8697-018
147,S-9666-030,B-8873,S-9666,30,11/27/2022,18:35:31,252,$283,8873/S-9666-030
148,S-5420-050,B-9949,S-5420,50,11/16/2022,22:35:55,105,$146,9949/S-5420-050


In [86]:
Items["Final Sell Price (O)"] = Items2["Final Sell Price (O)"]
Items

,Seller ID,Product ID,SP Combo,Item ID,Size,Order ID (O),Final Sell Price (O)
0,S-1442,36.0,S-1442-036,54.0,7,NaN,359
1,S-5393,33.0,S-5393-033,49.0,4,NaN,142
2,S-9229,35.0,S-9229-035,73.0,7,NaN,353
3,S-4532,19.0,S-4532-019,7.0,4,NaN,231
4,S-6313,8.0,S-6313-008,94.0,12,NaN,124
...,...,...,...,...,...,...,...
70,S-2488,39.0,S-2488-039,91.0,M,NaN,147
71,S-7929,41.0,S-7929-041,33.0,XS,NaN,NaN
72,S-6313,42.0,S-6313-042,34.0,M,NaN,145
73,S-1442,46.0,S-1442-046,70.0,S,NaN,NaN


### DONE!

* Time to export.
* Rows with no Final Sell Price meant that no bids were received and item is still available.
* Unfortunately, I had to manually input the Order ID in the Items to complete the table.

In [87]:
from pathlib import Path  
filepath = Path('C:/Users/AZhen/Downloads/Updated Items.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
Items.to_csv(filepath)